In [15]:
import os
import shutil
import numpy as np
import zarr
import yaml

from aicsimageio import AICSImage, writers
from ome_zarr.io import parse_url, ZarrLocation
from ome_zarr.writer import write_image

from matplotlib.pyplot import imshow
from tifffile import imwrite

import cv2


In [29]:
RTDIR = "/mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/"
GTDIR = "/mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/GT/"
TRDIR = "/mnt/efs/shared_data/restorators/monalisa_data/Actin_20nmScanStep/Input_no_interp/"

In [17]:
# utility functions
# define file path
def dataPath(
    dirPath: str = "/",
    fileName: str = None,
    fileID: int = 0,
    outputMode: int = 0  # output options: 0: file path, 1:splitted path, 2: file list
):

    fileList = os.listdir(dirPath)
    if fileName is None:
        fileName = fileList[fileID]
    fileName, filExt = os.path.splitext(fileName)
    filePath = os.path.join(dirPath, fileName + filExt)

    assert outputMode in [0, 1, 2], "outputMode must be 0 or 1"
    match outputMode:
        case 0:  # return file path
            return filePath
        case 1:  # return file path and file extension
            return dirPath, fileName, filExt
        case 2:  # return names in a file directory
            return fileList
        case _:
            raise ValueError(
                "outputMode must be 0-2 (0: file path, 1:splitted path, 2: file list")


# load data into an AICSImage object
def dataLoader(filepath: str = None):
    assert filepath != None, "filepath is None"
    assert os.path.exists(filepath), "file does not exist"
    img = AICSImage(filepath)
    return img


# save file into a zarr dircetory
def newZarr(
    pathZarr: str = "/",
    img: AICSImage = None,
    remove: bool = True
):

    assert img != None, "img is None"
    if os.path.exists(pathZarr):
        if remove:
            shutil.rmtree(pathZarr)
        else:
            raise OSError("file exists")
    os.mkdir(pathZarr)
    store = parse_url(pathZarr, mode="w").store
    image = img.data
    root = zarr.group(store=store)
    axes = img.dims.order.lower()
    imgShape = img.shape
    write_image(
        image=image,
        group=root,
        axes=axes,
        storage_options=dict(chunks=(1, imgShape[3], imgShape[4]))
    )


# convert all files in a directory to zarr
def zarrConverter(
    path: str = "/",
    remove: bool = True,
    dataTypes: str = ".tif"
):

    assert os.path.exists(path), "file does not exist"

    fileList = dataPath(path, outputMode=2)
    assert len(fileList) > 0, "file directory is empty"

    filePaths = []
    for file in fileList:
        if file.endswith(dataTypes):
            filePath = dataPath(path, file)
            dirPath, fileName, _ = dataPath(path, file, outputMode=1)
            zarrPath = os.path.join(dirPath, (fileName + '.zarr'))
            filePaths.append(zarrPath)

            img = dataLoader(filePath)
            newZarr(
                pathZarr=zarrPath,
                img=img,
                remove=remove
            )
    assert len(filePaths) > 0, "no files are converted"
    return filePaths

In [18]:
GTList = dataPath(GTDIR, outputMode=2)
TRList = dataPath(TRDIR, outputMode=2)

In [26]:
path = GTDIR + '/Edit3'
if not os.path.exists(path):
    os.mkdir(path)
for idx, fileName in enumerate(GTList):
    inputPath = os.path.join(GTDIR, fileName)
    if os.path.isfile(inputPath):
        if not os.path.exists(inputPath):
            img = AICSImage(inputPath)
            imwrite(os.path.join(path, fileName), img.data.squeeze()[0:2048, 0:2048])
        else:
            print(f"File {fileName} already exists")


File rec_c4_rec_CAM.tiff already exists
File c9_rec_CAM.hdf5.1.reconstruction.tiff already exists
File c9_rec_CAM_1.hdf5.1.reconstruction.tiff already exists
File c1_rec_CAM_1.hdf5.1.reconstruction.tiff already exists
File rec_c4_rec_CAM_9.tiff already exists
File c2_rec_CAM_1.hdf5.0.reconstruction.tiff already exists
File rec_c8_rec_CAM.tiff already exists
File rec_c8_rec_CAM_1.tiff already exists
File rec_c4_rec_CAM_5.tiff already exists
File rec_18h33m48s_rec_CAM.tiff already exists
File c5_rec_CAM_1.hdf5.1.reconstruction.tiff already exists
File rec_18h32m12s_rec_CAM.tiff already exists
File rec_c6_rec_CAM.tiff already exists
File rec_18h32m56s_rec_CAM.tiff already exists
File c3_rec_CAM.hdf5.1.reconstruction.tiff already exists
File rec_18h23m56s_rec_CAM.tiff already exists
File rec_c9_rec_CAM_2.tiff already exists
File rec_18h39m00s_rec_CAM.tiff already exists
File rec_18h25m14s_rec_CAM.tiff already exists
File c7_rec_CAM_1.hdf5.1.reconstruction.tiff already exists
File rec_c2_re

In [28]:
path = TRDIR + '/Edit2'
if not os.path.exists(path):
    os.mkdir(path)
else:
    print(f"Directory {fileName} already exists")

TRStack = []
for idx, fileName in enumerate(TRList):
    inputPath = os.path.join(TRDIR, fileName)
    outputPath = os.path.join(path, fileName)
    
    if os.path.isfile(inputPath):
        img = AICSImage(inputPath)
        imgUp = cv2.resize(img.data.squeeze()[0:1024, 0:1024], (2048, 2048), interpolation=cv2.INTER_CUBIC)
        TRStack.append(imgUp)
        if not os.path.exists(outputPath):
            imwrite(outputPath, imgUp)
            print(f"File {fileName} saved")


File rec_18h23m56s_rec_CAM_resliced_N=2.tiff saved
File rec_18h32m56s_rec_CAM_resliced_N=2.tiff saved
File rec_18h32m12s_rec_CAM_resliced_N=2.tiff saved
File rec_c6_rec_CAM_resliced_N=2.tiff saved
File rec_18h39m35s_rec_CAM_resliced_N=2.tiff saved
File rec_c11_rec_CAM_resliced_N=2.tiff saved
File rec_c2_rec_CAM_resliced_N=2.tiff saved
File c7_rec_CAM_1.hdf5.1.reconstruction_resliced_N=2.tiff saved
File rec_18h39m00s_rec_CAM_resliced_N=2.tiff saved
File rec_c9_rec_CAM_2_resliced_N=2.tiff saved
File rec_18h25m14s_rec_CAM_resliced_N=2.tiff saved
File c5_rec_CAM_1.hdf5.1.reconstruction_resliced_N=2.tiff saved
File rec_18h33m48s_rec_CAM_resliced_N=2.tiff saved
File rec_c4_rec_CAM_5_resliced_N=2.tiff saved
File rec_c8_rec_CAM_resliced_N=2.tiff saved
File rec_c4_rec_CAM_resliced_N=2.tiff saved
File c9_rec_CAM_1.hdf5.1.reconstruction_resliced_N=2.tiff saved
File c1_rec_CAM_1.hdf5.1.reconstruction_resliced_N=2.tiff saved
File rec_c4_rec_CAM_9_resliced_N=2.tiff saved
File c2_rec_CAM_1.hdf5.0.rec

In [ ]:
pathZarr = os.path.join(RTDIR + 'data.zarr')